# **DATATHON 2023: NTT-DATA CHALLENGE**

## **Requirements**:

In [15]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## **Main Program**

Read dataset

In [ ]:
df = pd.read_excel('../assets/consumo_material_clean.xlsx')
df.head()

Some preprocessing

In [ ]:
# Drop rows with NaN values
df = df.dropna()

# Split "ORIGEN" into "REGION", "HOSPITAL" and "DEPARTAMENTO"
df['ORIGEN'] = df['ORIGEN'].str.replace('--', '-')
df[['REGION', 'HOSPITAL', 'DEPARTAMENTO']] = df['ORIGEN'].str.split('-', expand=True)
df = df.drop(["ORIGEN"], axis=1)

# Categorical variables
categorical = ['CODIGO', 'PRODUCTO', 'NUMERO', 'REFERENCIA', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL']
df[categorical] = df[categorical].astype('category')

# Numeric variables
numerical_int = ['CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS']
numerical_float = ['PRECIO', 'IMPORTELINEA']
df[numerical_float] = df[numerical_float].astype('float')

# Date format
df['FECHAPEDIDO'] = pd.to_datetime(df['FECHAPEDIDO'], format='%d/%m/%y')
df['MES'] = df['FECHAPEDIDO'].dt.month
df['AÑO'] = df['FECHAPEDIDO'].dt.year
df = df.drop('FECHAPEDIDO', axis=1)

# Create new variable
df['PRECIOUNIDAD'] = df['IMPORTELINEA'] / df['CANTIDADCOMPRA']

df.to_csv('../assets/preprocessed_df.csv', index=False)

df.head()

In [ ]:
años = df['AÑO'].unique()
meses = df['MES'].unique()
productos = df['PRODUCTO'].unique()

# Crear todas las combinaciones posibles de año, mes y producto
todas_las_combinaciones = pd.DataFrame(list(itertools.product(años, meses, productos)), columns=['AÑO', 'MES', 'PRODUCTO'])

# Agrupar por año, mes y producto y realizar las operaciones de agregación
df_agrupado = df.groupby(['AÑO', 'MES', 'PRODUCTO']).agg({'CANTIDADCOMPRA': 'sum'}).reset_index()

# Combinar el DataFrame de todas las combinaciones con el DataFrame agrupado
# Usar un merge para asegurarse de que todas las combinaciones estén presentes
resultado_final = pd.merge(todas_las_combinaciones, df_agrupado, on=['AÑO', 'MES', 'PRODUCTO'], how='left')

# Rellenar los valores faltantes con 0 (o cualquier otro valor que sea apropiado)
resultado_final['CANTIDADCOMPRA'] = resultado_final['CANTIDADCOMPRA'].fillna(0)

resultado_final.to_csv('../assets/baiges_combinations.csv', index=False)


New dataset with groupby

In [ ]:
new_df = df[['PRODUCTO', 'AÑO', 'MES', 'TIPOCOMPRA', 'HOSPITAL', 'TGL', 'CANTIDADCOMPRA']].copy()

años = new_df['AÑO'].unique()
meses = new_df['MES'].unique()
productos = df['PRODUCTO'].unique()
tiposcompra = df['TIPOCOMPRA'].unique()
hospital = df['HOSPITAL'].unique()
tgl = df['TGL'].unique()

todas_las_combinaciones = pd.DataFrame(list(itertools.product(años, meses, productos,tiposcompra,hospital,tgl)), columns=['AÑO', 'MES', 'PRODUCTO','TIPOCOMPRA','HOSPITAL','TGL'])
df_agrupado = df.groupby(['AÑO', 'MES', 'PRODUCTO','TIPOCOMPRA','HOSPITAL','TGL']).agg({'CANTIDADCOMPRA': 'sum'}).reset_index()
resultado_final.to_csv('../assets/all_combinations.csv', index=False)



"""
new_df = new_df.groupby(['AÑO', 'MES', 'PRODUCTO', 'HOSPITAL', 'TIPOCOMPRA', 'TGL'], observed=True).agg({'CANTIDADCOMPRA': 'sum', 'UNIDADESCONSUMOCONTENIDAS': 'mean', 'PRECIO': 'mean', 'IMPORTELINEA': 'sum', 'PRECIOUNIDAD': 'mean'}).reset_index()

new_df.drop(['UNIDADESCONSUMOCONTENIDAS', 'PRECIO', 'IMPORTELINEA', 'PRECIOUNIDAD'], axis=1, inplace=True)

new_df.to_csv('../assets/new_df.csv', index=False)

new_df.head()
"""

Split train and test datasets

In [21]:
split_year = 2023
train = new_df.loc[new_df['AÑO'] < split_year]
test = new_df.loc[new_df['AÑO'] >= split_year]

In [22]:
train.to_csv('../assets/train.csv', index=False)
test.to_csv('../assets/test.csv', index=False)

ARIMA

In [ ]:
from pmdarima import auto_arima
from pmdarima.arima.utils import ndiffs
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.stattools import durbin_watson

# Carga de datos
data = pd.read_csv('new_df.csv')

# Agrupación de datos por producto y mes
grouped_data = data.groupby(['PRODUCTO', 'AÑO', 'MES']).agg({'CANTIDADCOMPRA': 'sum'}).reset_index()

# Inicializar un diccionario para almacenar los modelos
models = {}

# Función para verificar la estacionariedad
def check_stationarity(ts):
    return ndiffs(ts, test='adf') > 0

# Iterar sobre cada producto
for product_name in grouped_data['PRODUCTO'].unique():
    # Filtrar para un producto específico
    product_data = grouped_data[grouped_data['PRODUCTO'] == product_name]

    # Convertir a serie temporal
    product_series = product_data.pivot_table(values='CANTIDADCOMPRA', index=['AÑO', 'MES'], aggfunc='sum')

    # Dividir en conjunto de entrenamiento y prueba
    train = product_series[product_series.index.get_level_values('AÑO') < 2023]
    test = product_series[(product_series.index.get_level_values('AÑO') == 2023) & 
                          (product_series.index.get_level_values('MES') <= 10)]

    # Verificar si hay suficientes datos para el modelado
    if len(train) < 24 or not check_stationarity(train):
        print(f"No hay suficientes datos o la serie no es estacionaria para el producto {product_name}.")
        continue

    try:
        # Aplicar Auto ARIMA
        model = auto_arima(train, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)
        
        # Almacenar el modelo
        models[product_name] = model
    except Exception as e:
        print(f"Error al modelar el producto {product_name}: {e}")

# Preparar los datos de prueba
test_data = grouped_data[grouped_data['AÑO'] == 2023]
test_data = test_data[test_data['MES'] <= 10]
test_data = test_data.pivot_table(values='CANTIDADCOMPRA', index='PRODUCTO', columns='MES', aggfunc='sum')

# Realizar predicciones y comparar con los datos de prueba
all_predictions = []
all_actuals = []
for product_name, model in models.items():
    if product_name in test_data.index:
        # Número de meses a predecir
        months_to_predict = len(test_data.columns)
        
        # Realizar la predicción
        forecast = model.predict(n_periods=months_to_predict)
        all_predictions.extend(forecast)
        
        # Obtener los valores reales
        actual_values = test_data.loc[product_name].values
        all_actuals.extend(actual_values)

        # Gráfico de las predicciones para cada producto
        plt.figure(figsize=(10, 6))
        plt.plot(actual_values, label='Valores Reales', color='blue')
        plt.plot(forecast, label='Predicciones', color='red', linestyle='--')
        plt.title(f'Comparación de Valores Reales y Predicciones para {product_name}')
        plt.xlabel('Meses')
        plt.ylabel('Cantidad Comprada')
        plt.legend()
        plt.show()

# Cálculo de métricas de error globales
mse = mean_squared_error(all_actuals, all_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(all_actuals, all_predictions)

print(f"MSE Global: {mse}, RMSE Global: {rmse}, MAE Global: {mae}")

# Análisis de los residuos
residuos = np.array(all_actuals) - np.array(all_predictions)
dw_stat = durbin_watson(residuos)
print(f"Estadístico de Durbin-Watson: {dw_stat}")